# Did you tie your shoelaces this morning?

- https://adventofcode.com/2023/day/18

I've learned a new maths trick since solving day 10, and it involves tying your shoelaces. Or, rather, we can use the [Shoelace Formula](https://en.wikipedia.org/wiki/Shoelace_formula) to calculate the area of a polygon. In this formula, you pair up the X coordinate of one point with the Y coordinate of the next point, and vice versa, and take the difference between the products of the pairs. You do this for every consecutive pair of points, and the sum of all these differences is the area. Or, vice versa, you can also take the sums of all the even cross-paired coordinates and then subtract the odd cross-paired coordinates. Simple!

Not that we are actually asked for the area inside the polygon, we are asked for the area _plus the area of the lines themselves_, aka the boundary or perimeter. The area calculated actually includes half ot the line (picture the coordinates an the centres of the 1mx1m hole dug by the digger). To get the total area, we can apply [Pick's Theorem](https://en.wikipedia.org/wiki/Pick%27s_theorem) to figure out what part of the area $a$, calculated with the shoelace formula, is the integer $i$ interior, and by extension, what part of the $b$ boundary we then need to add to $a$ to get our $t$ total area:

$$
\begin{array}{ll}
a = i + \frac{b}{2} - 1
\\ \to i = a - \frac{b}{2} + 1
\\
\\
t = i + b
\\ \to t = a + 1 - \frac{b}{2} + b
\\ \to t = a + 1 + \frac{b}{2}
\end{array}
$$

Note that $b$ is simply the sum of the numbers in the instructions.

Of course, we'll have to map those dig instructions to coordinates first, but that's just a matter of adding to a running coordinate value with each instruction vector; something that [`numpy.cumsum()`](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html#numpy-cumsum) takes care of for us. I've added in the length of the dig instruction as a 3rd column so I can extract the boundary length from the final row.

I'm ignoring the colour value entirely, I'm sure that'll come to play for part 2.


In [1]:
from __future__ import annotations

import typing as t

import numpy as np

type Coords = np.ndarray[tuple[int, t.Literal[2]], np.dtype[np.int_]]


def parse(line: str) -> tuple[int, int, int]:
    dir, count, _ = line.split()
    count = int(count)
    match dir:
        case "R":
            return (count, 0, count)
        case "D":
            return (0, count, count)
        case "L":
            return (-count, 0, count)
        case "U":
            return (0, -count, count)
        case _:
            raise ValueError(f"Unknown dig direction {dir} on line {i}")


def dig_instructions(
    instructions: list[str], parse: t.Callable[[str], tuple[int, int, int]] = parse
) -> tuple[Coords, int]:
    accumulated = np.cumsum(np.array([parse(line) for line in instructions]), axis=0)
    return accumulated[:, :2], accumulated[-1, -1]


def shoelace(coords: Coords) -> int:
    x, y = coords[:, 0], coords[:, 1]
    odd = np.dot(x, np.roll(y, 1))
    even = np.dot(y, np.roll(x, 1))
    return 0.5 * np.abs(odd - even)


def dug_area(coords: Coords, boundary: int) -> int:
    area = int(shoelace(coords))
    return area + 1 + boundary // 2


test_digplan = """\
R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
"""


coords, boundary = dig_instructions(str(test_digplan).splitlines())
assert dug_area(coords, boundary) == 62

In [2]:
import aocd

digplan = aocd.get_data(day=18, year=2023).splitlines()
print("Part 1:", dug_area(*dig_instructions(digplan)))

Part 1: 48400


# Someone was colour _and_ number blind?

Here we go, part two reveals that the size of the polygon is vastly larger than part 1 would have led us to believe. The colour value is actually an encoded dig instruction. Good thing we can use maths to get the area still!

I refactored the `dig_instructions()` function to take a `parse` callable, so that part 2 can pass in a different parser.


In [3]:
def corrected_parse(line: str) -> tuple[int, int, int]:
    colour = line.rpartition("(#")[-1].rstrip(")")
    count = int(colour[:5], 16)
    match colour[5]:
        case "0":  # R
            return (count, 0, count)
        case "1":  # D
            return (0, count, count)
        case "2":  # L
            return (-count, 0, count)
        case "3":  # U
            return (0, -count, count)
        case _:
            raise ValueError(f"Unknown dig direction {dir} on line {i}")


coords, boundary = dig_instructions(str(test_digplan).splitlines(), corrected_parse)
assert dug_area(coords, boundary) == 952408144115

In [4]:
print("Part 1:", dug_area(*dig_instructions(digplan, corrected_parse)))

Part 1: 72811019847283
